In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import GridSearchCV
from yellowbrick.model_selection import FeatureImportances


RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

# Introduction:

DataSet: https://www.kaggle.com/uciml/pima-indians-diabetes-database

The goal of this project is to develop a Machine Learning model capable of predicting if a woman has diabetes, with some parameters, such as glucose in the last blood exam or insulin. This model will be a classifier and it will be tested through some different alternatives to check which one has the best outcome. 

Dependent variable: Outcome

Examples of Independent Variables: Life expectancy and Poverty rates

In [ ]:
diabetes = pd.read_csv('diabetes.csv')

diabetes

# First look at the database

In [ ]:
diabetes.info()

We can learn through the info function that there are no null values in the database, therefore we don't need to address any solution to the missing values

In [ ]:
diabetes.describe()

In [ ]:
diabetes.hist(bins=50, figsize=(20, 15))
plt.show()

### Missing Values

As we can see, the dabatabase don't have null values, but, there are multiple 0s. This would make sense in some cases, such as pregnancies, but in others, for example, inslulin and blood pressure. This situations needs to be addressed. The approach we'll take will be by replacing the mean values of the whole column to the missing value. 

# Dividing in Train Test

We will be making the division of the train and test set stratifically. This is due to the fact that the parameter "Diabetes Pedigree Function" is believed to have a huge impact on the outcome, therefore, we believe that the train and the test sets should have the same proportion. Also, as we have a small amount of samples (m = 768) this type of separation can be usefull since, if we were to not to use it, the proportion would be unequal.

In [ ]:
q = diabetes['DiabetesPedigreeFunction'].quantile([0.2,0.4,0.6,0.8]).values

def check_pedigree(x):
    result = 0
    for val in q:
        if x > val:
            result += 1
        else:
            break
    return result
diabetes['pedigree_cat'] = diabetes['DiabetesPedigreeFunction'].apply(check_pedigree)
diabetes['pedigree_cat'].value_counts(True)

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit

split = StratifiedShuffleSplit(
    n_splits=1,
    test_size=0.2,
    random_state=RANDOM_SEED,
)
for train_index, test_index in split.split(diabetes, diabetes['pedigree_cat']):
    strat_train_set = diabetes.loc[train_index]
    strat_test_set = diabetes.loc[test_index]

In [ ]:
print(strat_train_set['pedigree_cat'].value_counts(True), '\n', strat_test_set['pedigree_cat'].value_counts(True))

As we can see, both test and train has the same proportions of values.

In [ ]:
#Now we need to drop the created column
strat_train_set.drop(['pedigree_cat'], axis=1, inplace=True)
strat_test_set.drop(['pedigree_cat'], axis=1, inplace=True)

# Exploratory analysis

In [ ]:
diabetes_train = strat_train_set.copy()

In [ ]:
plt.figure(figsize=(16, 8))
sns.set(style="whitegrid")
corr = diabetes_train.corr()
sns.heatmap(corr,annot=True,cmap="coolwarm")

In [ ]:
plt.figure(figsize=(16, 8))
sns.set(style="whitegrid")
corr = diabetes_train.corr(method='kendall')
sns.heatmap(corr,annot=True,cmap="coolwarm")

In [ ]:
plt.figure(figsize=(16, 8))
sns.set(style="whitegrid")
corr = diabetes_train.corr(method='spearman')
sns.heatmap(corr,annot=True,cmap="coolwarm")

As we can see, the kendall correlation seems to be the one with the higher values for the outcome relation with other variables. This indicates that the relationship between the outcome and other variables may not be linear.

We can see that the correlation between age and pregnancies is the highest. Also, the outcome appears to have some kind of relation with glucose.

# Separating Independent and Dependent Variables

In [ ]:
#Replacing the 0s with Nan so that SimpleImputer can operate
strat_train_set = strat_train_set.replace(to_replace={
             'BloodPressure':{0:np.nan}, 
             'Insulin':{0:np.nan},
             'SkinThickness':{0:np.nan},
             'BMI':{0:np.nan},
             'Glucose':{0:np.nan},
                 })
strat_train_set.dropna(subset=['BMI', 'BloodPressure', 'Glucose'], inplace = True)

diabetes_train = strat_train_set.drop('Outcome', axis=1)

diabetes_labels = strat_train_set['Outcome'].copy()

## Replacing Invalid values

In [ ]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer


diabetes_in = diabetes_train.copy()

imputer = IterativeImputer(max_iter=10, random_state=0)

imputer.fit(diabetes_in)

In [ ]:
temp = imputer.transform(diabetes_in)

diabetes_inputed = pd.DataFrame(temp, columns=diabetes_in.columns)

diabetes_inputed.hist(bins=50, figsize=(20, 15))
plt.show()

As we can see, there are no more 0s values, although we have huge peeks in the mean values of some parameters, this makes the database much better

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

scaler.fit(diabetes_inputed)

temp = scaler.transform(diabetes_inputed)

diabetes_train = pd.DataFrame(temp, columns=diabetes_inputed.columns)
diabetes_train

# Binary Classifier

In [ ]:
from sklearn.linear_model import SGDClassifier

# Existe aleatoriedade dentro do SGDClassifier, por isso o argumento
# random_state=RANDOM_SEED.
sgd_clf = SGDClassifier(
    max_iter=500,
    tol=1e-3,
    random_state=RANDOM_SEED,
)
sgd_clf.fit(diabetes_train, diabetes_labels)

### Cross Validation

Before we test the performance of the Model "SGD Classifier" we should know the proportions of the target variable. The reason for this is because if the data has a proportion of 80% of people who have diabetes and 20% who haven´t, and the Classifier has an accuracy of 80%, the model would be as bad as one who just guess constanly "user with diabetes".

In [ ]:
diabetes_labels.value_counts(True)

In [ ]:
import time
from sklearn.model_selection import cross_val_score

t1 = time.process_time()
Binary_score = cross_val_score(
               sgd_clf,
               diabetes_train,
               diabetes_labels,
               cv=10,
               scoring="accuracy",
               n_jobs=-1,
               )
t2 = time.process_time()

print(Binary_score)

Here we see that the model had an accuracy around 73%, which may seem ok (but not good enough). But, when it compares the porportion of people who has diabetes in the data, we realize that this model was almost as bad as one who keeps only guessing "people who don't have diabetes". Therefore, we need to improve.

## Hyperparameters adjusting

It is possible to further improve the performance of the SGDC Classifier by using GridSearch, which combines thousands of possible parameters of the model and finds the best of them. For that, it is necessary to pass a grid of parameters, it is necessary to know the SGDC classifier parameters.

In [ ]:
sgd_clf.get_params().keys()

In [ ]:
# train across 5 folds, that's a total of (6+4)*5=50 rounds of training.
param_grid = {
    'alpha': [1e-4, 1e-3, 1e-2, 1e-1, 1e0, 1e1, 1e2, 1e3], # learning rate
    'loss': ['log'], # logistic regression,
    'penalty': ['l2'],
    'n_jobs': [-1]
}

sgd_clf_grid = GridSearchCV(
    sgd_clf,  # Modelo
    param_grid,  # Grid
    cv=5,  # Partições de C.V.
    scoring='neg_mean_squared_error',
    return_train_score=True,
    n_jobs=-1,
)

Now, the best SGDC classifier was obtained (that is, the model with the best parameters). With that said, it is necessary to train the model again.

In [ ]:
sgd_clf_grid.fit(diabetes_train, diabetes_labels)

In [ ]:
sgd_clf_grid.best_params_

After using Grid Search, it is possible to calculate once again the model accuracy, and a higher performance can be observed.

In [ ]:
t1 = time.process_time()
Binary_score_grid = cross_val_score(
    sgd_clf_grid,
    diabetes_train,
    diabetes_labels,
    cv=10,
    scoring="accuracy",
    n_jobs=-1,
)
t2 = time.process_time()

print(Binary_score_grid)

## Feature importance

Lastly, it is also interesting to analyze the features that impacted the most on the prediction of the model. For that, the graphic below was plotted and Insulin was the feature that had the most importance, followed by Glucose and Blood Pressure, the latter having a negative coefficient value, however the magnitude is what matters, and the negative coefficient only indicates that the dependent variable and independent variable have a strong negative correlation. That makes sense, since people with lower Blood Pressure tend to not have diabetes, it is something more predictable. However, when this value gets higher, more outliers can appear, and it becomes more difficult to predict whether a person will have diabetes or not. 

In [ ]:
viz = FeatureImportances(sgd_clf, relative=False, topn=3)
viz.fit(diabetes_train, diabetes_labels)
viz.show()

Also, by plotting the importance of the all features, the difference of importance between the three features discussed previously and the reamining features is quite notable.

In [ ]:
viz = FeatureImportances(sgd_clf, relative=False)
viz.fit(diabetes_train, diabetes_labels)
viz.show()

### Confusion Matrix

Although the accuracy wasn´t really good, it is important to understand in which cases the model has missed. With the confusion matrix it was learned that there was a lot of false negatives. This is unacceptable in this scneario, since the model should never overlook people with diabetes because it is a health situation and people can be harmed if they are not properly diagnosed. Therefore, the model should focous on having a high Recall.

In [ ]:
from sklearn.model_selection import cross_val_predict

y_train_pred = cross_val_predict(sgd_clf_grid, diabetes_train, diabetes_labels, cv=3, n_jobs=-1)

In [ ]:
from sklearn.metrics import confusion_matrix

mat = confusion_matrix(diabetes_labels, y_train_pred)
mat

### Precision Recall

In [ ]:
from sklearn.metrics import precision_score, recall_score

print(precision_score(diabetes_labels, y_train_pred))
print(recall_score(diabetes_labels, y_train_pred))

As we can see, the Recall is actually low, and now it is important to adjust this value to make it more accpetable.

In [ ]:
y_scores = cross_val_predict(
    sgd_clf_grid,
    diabetes_train,
    diabetes_labels,
    cv=3,
    method="decision_function",
    n_jobs=-1,
)

In [ ]:
from sklearn.metrics import precision_recall_curve

precisions, recalls, thresholds = precision_recall_curve(diabetes_labels, y_scores)

plt.figure(figsize=(8, 4))

plt.plot(thresholds, precisions[:-1], "b--", label="Precision", linewidth=2)
plt.plot(thresholds, recalls[:-1], "g-", label="Recall", linewidth=2)

plt.xlabel("Threshold", fontsize=16)
plt.legend(loc="upper left", fontsize=16)
plt.xlim([-5, 5])
plt.ylim([0, 1])
plt.show()

With this graph it is possible to discover a value of Threshold that increases the Recall, which is -0.75. With this, it is possible to adjust the results for the aim of decreasing false negatives

In [ ]:
y_train_pred_90 = (y_scores > -0.75)

In [ ]:
print('Precision: {}'.format(precision_score(diabetes_labels, y_train_pred_90)))
print('Recall: {}'.format(recall_score(diabetes_labels, y_train_pred_90)))

In [ ]:
plt.figure(figsize=(8, 6))

plt.plot(recalls, precisions, "b-", linewidth=2)

plt.xlabel("Recall", fontsize=16)
plt.ylabel("Precision", fontsize=16)
plt.axis([0, 1, 0, 1])

plt.show()

It is possible to see that when the Recall is increased, the Precision has the opposite reaction. In the current scneario, this is not a problem, because the goal is to just increase the Recall.

#### ROC Curve

In [ ]:
from sklearn.metrics import roc_curve

fpr, tpr, thresholds = roc_curve(diabetes_labels, y_scores)

plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, linewidth=2)
plt.plot([0, 1], [0, 1], 'k--')
plt.axis([0, 1, 0, 1])
plt.xlabel('False Positive Rate', fontsize=16)
plt.ylabel('True Positive Rate', fontsize=16)
plt.show()

For the diabetes problem, the ideal format of this curve is when the area below the curve is high thus making the curve closer to the upper left corner. With the SGD classifier, the curve obtained does not match the ideal shape. This concept is known as ROC curve and what is desired in the present study is a high sensitivity, or true positive rate, meaning a high recall.

# Random Forest Classifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier

forest_clf = RandomForestClassifier(n_estimators=100, random_state=42)


t1 = time.process_time()
Rand_Forrest_Score = cross_val_score(
    forest_clf,
    diabetes_train,
    diabetes_labels,
    cv=10,
    scoring="accuracy",
    n_jobs=-1,
)
t2 = time.process_time()

print(Rand_Forrest_Score)

Through the cross val score, it seems that this classifier has a better perfomance than the SDG.

## Adjusting Hyperparameters 

Similarly to the SGDC model, here the hyperparameters of the model will be adjusted.

In [ ]:
forest_clf.get_params().keys()

In [ ]:
param_grid = {
    'bootstrap': [True],
    'max_depth': [80, 90, 100, 110],
    'max_features': [2, 3],
    'min_samples_leaf': [3, 4, 5],
    'min_samples_split': [8, 10, 12],
}

forest_clf_grid = GridSearchCV(
    forest_clf,  # Modelo
    param_grid,  # Grid
    cv=5,  # Partições de C.V.
    scoring='neg_mean_squared_error',
    return_train_score=True,
    n_jobs=-1,
)

In [ ]:
forest_clf_grid.fit(diabetes_train, diabetes_labels)

In [ ]:
forest_clf_grid.best_params_

Now, with the hyperparameters adjusted, there is a slight improvement to the model's performance.

In [ ]:
t1 = time.process_time()
Random_Forrest_grid_Score = cross_val_score(
    forest_clf_grid,
    diabetes_train,
    diabetes_labels,
    cv=10,
    scoring="accuracy",
    n_jobs=-1,
)
t2 = time.process_time()

print(Random_Forrest_grid_Score)

## Feature Importances

By plotting the features importance graphs, there are some noticable differences:

In [ ]:
viz = FeatureImportances(forest_clf, relative=False, topn=3)
viz.fit(diabetes_train, diabetes_labels)
viz.show()

In [ ]:
viz = FeatureImportances(forest_clf, relative=False)
viz.fit(diabetes_train, diabetes_labels)
viz.show()

- Although Glucose still is one of the most important features, now the 3rd best is no longer Blood pressure, it is BMI. In fact, Blood Pressure is the least important feature, indicating the difference of impact in comparison to the previous model.

- Blood Pressure now doesn't have a negative coefficent, which means that for the Random Forest higher values are more important to the prediction than lower values, contrary to what happened in SGDC Classifier.

- Unlike previsously, now age has a bigger impact on the prediction. This corroborates what the hypothesis made in the exploratory analysis, that age was important to determine if a person will have diabetes.

In [ ]:
# O "score" vai ser a probabilidade de que a amostra seja da classe positiva.
y_probas_forest = cross_val_predict(
    forest_clf_grid,
    diabetes_train,
    diabetes_labels,
    cv=3,
    method="predict_proba",
    n_jobs=-1,
)

# Gambiarra para desviar do bug #9589 introduzido no Scikit-Learn 0.19.0:
y_scores_forest = y_probas_forest[:, 1]

In [ ]:
fpr_forest, tpr_forest, thresholds_forest = roc_curve(
    diabetes_labels,
    y_scores_forest,
)

plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, "r:", linewidth=2, label="SGD")
plt.plot(fpr_forest, tpr_forest, linewidth=2, label="Random Forest")
plt.plot([0, 1], [0, 1], 'k--')
plt.axis([0, 1, 0, 1])
plt.xlabel('False Positive Rate', fontsize=16)
plt.ylabel('True Positive Rate', fontsize=16)
plt.legend(loc="lower right", fontsize=16)
plt.show()

In [ ]:
from sklearn.metrics import roc_auc_score

print('SGD: {:4f}'.format(roc_auc_score(diabetes_labels, y_scores)))
print('RandomForest: {:4f}'.format(roc_auc_score(diabetes_labels, y_scores_forest)))

As it can be seen, the Random Forest Classifier has a better performance because the area under the curve is higher.

# Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_predict

clf_log = LogisticRegression(random_state=RANDOM_SEED)


t1 = time.process_time()
Log_Score = cross_val_score(
    clf_log,
    diabetes_train,
    diabetes_labels,
    cv=10,
    scoring="accuracy",
    n_jobs=-1,
)
t2 = time.process_time()

print(Log_Score)

## Adjusting Hyperparameters

In [ ]:
clf_log.get_params().keys()

In [ ]:
param_grid = [
    {
    'penalty' : ['l1', 'l2'],
    'C' : np.logspace(-4, 4, 20),
    'solver' : ['liblinear']
    },
]

logistic_clf_grid = GridSearchCV(
    clf_log,  # Modelo
    param_grid,  # Grid
    cv=5,  # Partições de C.V.
    scoring='neg_mean_squared_error',
    return_train_score=True,
    n_jobs=-1,
)

In [ ]:
logistic_clf_grid.fit(diabetes_train, diabetes_labels)

In [ ]:
logistic_clf_grid.best_params_

## Features Importance

After analysing the importance of features in the logistic regression model, it is noticable that glucose still remains as the most important feature, and pregnancies also becomes the 2nd most important (like in the SGDC classifier). BMI, like the Random Forest model, also becomes one of the top three most important features. The higher the parameter BMI is, the more likely a person is to develop diabtes, according to a study (https://vitagene.com/blog/does-obesity-cause-type-2-diabetes/), therefore it makes sense that this would be one of the most important features.

In [ ]:
viz = FeatureImportances(clf_log, relative=False, topn=3)
viz.fit(diabetes_train, diabetes_labels)
viz.show()

In [ ]:
viz = FeatureImportances(clf_log, relative=False)
viz.fit(diabetes_train, diabetes_labels)
viz.show()

In [ ]:
# O "score" vai ser a probabilidade de que a amostra seja da classe positiva.
y_probas_logistic = cross_val_predict(
    logistic_clf_grid,
    diabetes_train,
    diabetes_labels,
    cv=3,
    method="predict_proba",
    n_jobs=-1,
)

# Gambiarra para desviar do bug #9589 introduzido no Scikit-Learn 0.19.0:
y_scores_logistic = y_probas_forest[:, 1]

In [ ]:
Log_Score_Grid = cross_val_score(
    logistic_clf_grid,
    diabetes_train,
    diabetes_labels,
    cv=10,
    scoring="accuracy",
    n_jobs=-1,
)
Log_Score_Grid

After using the Logistic Regression it is possible to conclude that its performance is better than the SGD classifier, with an accuracy of 76%. 

# SVM

In [ ]:
from sklearn.svm import LinearSVC, SVC

clf_svm= SVC(C=1e5, kernel='linear', degree=1,  random_state=RANDOM_SEED)
clf_svm.fit(diabetes_train, diabetes_labels)

In [ ]:
%%time
from sklearn.model_selection import GridSearchCV, ShuffleSplit
from pprint import pprint

grid = GridSearchCV(
    clf_svm,
    {
        'C': [10.0**k for k in np.arange(-3, 5, 2)], 'kernel': ('linear', 'rbf'),
        'degree': [1, 2, 3, 4, 5, 6]
    },
    scoring='accuracy',
    cv=ShuffleSplit(
        n_splits=100,
        test_size=0.33,
        random_state=RANDOM_SEED,
    ),
    n_jobs=-1,
    verbose=3,
)
# len(clf_svm.get_params().keys())
grid.fit(diabetes_train, diabetes_labels)

print(grid.best_params_)

In [ ]:
P = np.array([p['C'] for p in grid.cv_results_['params']])
M = grid.cv_results_['mean_test_score']
S = grid.cv_results_['std_test_score']

for p, m, s in zip(P, M, S):
    print(f'C = {p:.5f}: mean_accuracy = {m:.3f}, stddev_accuracy = {s:.3f}')

plt.figure(figsize=(8, 6))
plt.errorbar(P, M, S, capsize=4)
plt.semilogx()
plt.title('Accuracy from CV', fontsize=20)
plt.xlabel(r"$C$", fontsize=20)
plt.ylabel(r"$accuracy$", fontsize=20)
plt.show()

In [ ]:
#General accuracy of SVM
M.mean()

In [ ]:
import random
M_set = set(M)
M_list = list(M_set)
M_list.append(M[random.randint(0,len(M))])
M_list.append(M[random.randint(0,len(M))])
M_list.append(M[random.randint(0,len(M))])

In [ ]:
len(M_list)

The idea of SVM algorithm is to trace a curve that can separate when a patient has diabets. However, sometimes the samples are mixed and a curve that completely separates those values is not possible. To calculate the best curve possible, the C parameter used in the algorithm associate penalties to the values that are miscalculated. The higher this parameter is, the less regularization the model will execute, and, hence, overfitting will occur.

This model also has another parameter degree. This determines the degree of the curve that will be used to separate the elements. Whilst creating the model, the grid search was used with this parameter (and also with C), and the outcome was that the model had a better performance when degree was equal to 1. Therefore, the curve obtained is linear.

Finally, it is discoverded that the general accuracy of the Supporting Vector Machine is the diabetes scenario has a 72%  of accuracy.

## Features Importance

In [ ]:
def f_importances(coef, names):
    imp = coef
    imp,names = zip(*sorted(zip(imp,names)))
    plt.barh(range(len(names)), imp, align='center')
    plt.yticks(range(len(names)), names)
    plt.show()

f_importances(clf_svm.coef_[0], diabetes_train.columns.tolist())

After analysing the importance of features in the Supporting Vector Machine model, it is noticable that glucose still remains as the most important feature, and pregnancies also becomes the 2nd most important (similary to the previous models).

# Conclusion

In [ ]:
pd.DataFrame({
    'Logistic Regression': Log_Score_Grid,
    'Random Forrest': Random_Forrest_grid_Score,
    'Binary Classifier': Binary_score_grid,
    'SVM': M_list,
}).plot.box(
    xlabel='Regressor',
    ylabel=r'Accuracy $[\mathtt{USD}]$',
    figsize=(10, 5),
);

Analyzing the general results of accuracy of each model, it is possible to understand that all of them had a similar performance. But analyzing the details it is possible to conclude that Random Forrest had the best general accuracy. Moreover, it is intersting to point out that a specific outlier of Logistic Regression, Random Forrest and Binary Classifier had aproximalary the same accuracy. This may have been due to the fact that all of them had the same cut of the cros val and that section of the database impact the best it could to predict if semeone had or not diabetes.

In [ ]:
from sklearn.metrics import accuracy_score

final_model = forest_clf_grid.best_estimator_

X_test = strat_test_set.drop('Outcome', axis=1)
y_test = strat_test_set['Outcome'].copy()

X_test_prepared = imputer.transform(X_test)
X_test_prepared = scaler.transform(X_test_prepared)
final_predictions = final_model.predict(X_test_prepared)

final_accuracy = accuracy_score(y_test, final_predictions)

print(f'Accuracy = {final_accuracy}')

In [ ]:
y_test.value_counts(True)

**In Conclusion**, the Random Forrest Classifier perfomed a marvelous job, since it had an accuracy of 70,77% in the test set. This shows that not only it doesn't overfit or underfit but also acts much better then a model which just guess that everybody doesn't have diabetes. This is because in the cell above, it is learned that there were 61,03% of people who didn't have diabetes. Therefore a model that only guess, would score an 61,03% of accuracy and the Random Forrest one was 10% better.